In [1]:
import tensorflow as tf
from pymagnitude import *
import os
import csv
from tqdm import tqdm
import logging
from collections import Counter
import pickle
import re
import regex
import logging
import numpy as np

# QNLI training data path
TRAINING_DATAFILE = "/Users/zxq001/QNLI/train.tsv"
# load pretrained embedding
vectors = Magnitude("/Users/zxq001/glove.840B.300d.magnitude")

# the maximimum length for the question sequence
MAX_SEQ_LENGTH = 100

# u = question sequence embedding (MAX_SEQ_LENGTH, 300) -> 1500D bidirectional LSTM -> maxpooling
q_in = tf.keras.layers.Input(shape=(MAX_SEQ_LENGTH, vectors.dim))
q_Bidir_LSTM = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(1500, activation='tanh', return_sequences=True), merge_mode='concat')(q_in)
expanded_q_LSTM = tf.keras.layers.Lambda(lambda x: tf.expand_dims(x, axis=-1))(q_Bidir_LSTM)
q_maxpool = tf.keras.layers.MaxPooling2D(pool_size=(MAX_SEQ_LENGTH, 1))(expanded_q_LSTM)
u =  tf.keras.layers.Lambda(lambda x: tf.squeeze(x, axis=-1))(q_maxpool)

# v = answer sequence embedding (MAX_SEQ_LENGTH, 300) -> 1500D bidirectional LSTM -> maxpooling
a_in = tf.keras.layers.Input(shape=(MAX_SEQ_LENGTH, vectors.dim))
a_Bidir_LSTM = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(1500, activation='tanh', return_sequences=True), merge_mode='concat')(a_in)
expanded_a_LSTM = tf.keras.layers.Lambda(lambda x: tf.expand_dims(x, axis=-1))(a_Bidir_LSTM)
a_maxpool = tf.keras.layers.MaxPooling2D(pool_size=(MAX_SEQ_LENGTH, 1))(expanded_a_LSTM)
v =  tf.keras.layers.Lambda(lambda x: tf.squeeze(x, axis=-1))(a_maxpool)

Instructions for updating:
Colocations handled automatically by placer.


In [4]:
u-v

<tf.Tensor 'sub:0' shape=(?, 1, 3000) dtype=float32>

In [3]:
v

<tf.Tensor 'lambda_3/Squeeze:0' shape=(?, 1, 3000) dtype=float32>

In [39]:
# trains and saves the model
def train():
    # load data
    questions, answers, labels = read_QNLI_dataset(TRAINING_DATAFILE)
    
    # embedding transformation
    X1 = vectors.query(questions)
    X2 = vectors.query(answers)
    y = labels
    
    # concatnate input vectors
    X = tf.concat([X1, X2, abs(X1-X2), tf.math.multiply(X1, X2)], 1)
    
    # create model
    q = tf.keras.layers.Input(shape=(None,))
    a = tf.keras.layers.Input(shape=(None,))
    
    qembeddings = tf.keras.layers.Embedding(len(vectors), vectors.dim, mask_zero=True)(q)
    aembeddings = tf.keras.layers.Embedding(len(vectors), vectors.dim, mask_zero=True)(a)
    
    qlstm1 = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(vectors.dim, return_sequences=True))(qembeddings)
    qlstm = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(vectors.dim))(qlstm1)
    
    alstm1 = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(vectors.dim, return_sequences=True))(aembeddings)
    alstm = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(vectors.dim))(alstm1)
    
    qmaxpool = tf.keras.layers.GlobalMaxPooling1D()(qlstm)
    amaxpool = tf.keras.layers.GlobalMaxPooling1D()(alstm)
    
    
    output = tf.keras.layers.Dense(1, activation="softmax")(maxpool)
    model = tf.keras.Model(inputs=i, outputs=output)
    

In [5]:
# read QNLI dataset from GLUE benchmark
def read_QNLI_dataset(FILE_PATH):
    trainfile = open(FILE_PATH)
    trainfile = csv.reader(trainfile, delimiter='\t')
    questions_raw = []
    answers_raw = []
    labels = []
    # skip the header
    next(trainfile)
    for row in trainfile:
        questions_raw.append(row[1])
        answers_raw.append(row[2])
        if row[3] == "entailment":
            labels.append(1)
        else:
            labels.append(0)
    # seperate the questions and answers into words
    questions = []
    answers = []
    for question in questions_raw:
        questions.append(regex.findall(r"[^[:punct:] ]+|[[:punct:]]", question))
    for answer in answers_raw:
        answers.append(regex.findall(r"[^[:punct:] ]+|[[:punct:]]", answer))
    return questions, answers, labels

In [6]:
questions, answers, labels = read_QNLI_dataset(TRAINING_DATAFILE)